In [17]:
import networkx as nx
import numpy as np
import pandas as pd
from scipy.spatial import distance_matrix
from queue import PriorityQueue
import time

In [2]:
!unzip DATA.zip

Archive:  DATA.zip
   creating: DATA/
  inflating: DATA/Roanoke.tsp        
  inflating: DATA/NYC.tsp            
  inflating: DATA/solutions.csv      
  inflating: DATA/UKansasState.tsp   
  inflating: DATA/Champaign.tsp      
  inflating: DATA/UMissouri.tsp      
  inflating: DATA/Boston.tsp         
  inflating: DATA/Atlanta.tsp        
  inflating: DATA/Cincinnati.tsp     
  inflating: DATA/Denver.tsp         
  inflating: DATA/SanFrancisco.tsp   
  inflating: DATA/Toronto.tsp        
  inflating: DATA/Philadelphia.tsp   
  inflating: DATA/Berlin.tsp         


In [3]:
instance='Atlanta'
n=int(open('DATA/'+instance+'.tsp').readlines()[2][len('DIMENSION: '):])
input=np.loadtxt("DATA/"+instance+".tsp",skiprows=5,max_rows=n)
print(n)

20


In [9]:
# initial solution from mst heuristic
G = nx.from_numpy_matrix(np.matrix.round(distance_matrix(input[:,1:],input[:,1:])))
T=nx.minimum_spanning_tree(G)
eulerian_cycle = list(nx.eulerian_circuit(T.to_directed(),source=0))
visited_nodes=[]
tour_nodes=[]
for edge in eulerian_cycle:
  if(edge[0] not in visited_nodes):
    tour_nodes.append(edge[0])
  visited_nodes.append(edge[0])
tour_nodes.append(tour_nodes[0])
print(tour_nodes)
print(nx.path_weight(G,tour_nodes,'weight'))

[0, 6, 1, 2, 16, 12, 8, 11, 10, 19, 5, 4, 17, 13, 9, 15, 3, 7, 14, 18, 0]
2415132.0


In [21]:
# branch and bound from 
G = nx.from_numpy_matrix(np.matrix.round(distance_matrix(input[:,1:],input[:,1:])))
q = PriorityQueue()
start_node=0
q.put((np.infty,[start_node]))
q.put((nx.path_weight(G,tour_nodes,'weight'),tour_nodes))
best=(nx.path_weight(G,tour_nodes,'weight'),tour_nodes)
timeout = time.time() + 60*5   # 5 minutes from now
while not q.empty():
    if time.time() > timeout:
        break
    item = q.get()
    print(item)
    for node in G.nodes-item[1]:
      path=item[1].copy()
      path.append(node)
      if len(G.nodes-path)==0:
        path = path+[path[0]]
        cost = nx.path_weight(G,path,'weight')
        if cost < best[0]:
          best=(cost,path)
          print('new best:')
          print(best)
      else:
        subproblem=G.copy()
        subproblem.remove_nodes_from(path)
        mst=nx.minimum_spanning_tree(subproblem)
        mst_cost=0
        for edge in mst.edges.data():
          mst_cost = mst_cost + edge[2]['weight']
        path_cost=nx.path_weight(G,path,'weight')
        min_weight_edge_start = min(G.edges(path[0]), key=lambda x: G.get_edge_data(x[0], x[1])["weight"])
        min_weight_edge_start = G.get_edge_data(min_weight_edge_start[0],min_weight_edge_start[1])['weight']
        min_weight_edge_end = min(G.edges(path[len(path)-1]), key=lambda x: G.get_edge_data(x[0], x[1])["weight"])
        min_weight_edge_end = G.get_edge_data(min_weight_edge_end[0],min_weight_edge_end[1])['weight']
        connection_cost = min_weight_edge_start + min_weight_edge_end
        cost = mst_cost + path_cost + connection_cost
        if cost < best[0]:
          q.put((cost,path))

Streaming output truncated to the last 5000 lines.
(1722803.0, [0, 6, 11, 8, 12, 16, 1, 2, 17, 10, 19, 4, 13, 9, 15])
(1740535.0, [0, 6, 2, 8, 11, 1, 12, 16, 10, 19, 4, 17])
(1740540.0, [0, 6, 11, 12, 1, 16, 2, 8, 4, 10])
(1691803.0, [0, 6, 11, 12, 1, 16, 2, 8, 4, 10, 19])
(1740542.0, [0, 6, 1, 12, 2, 16, 8, 11, 10, 4, 17, 7])
(1740544.0, [0, 6, 1, 2, 12, 16, 11, 8, 10, 4, 19, 5])
(1740552.0, [0, 6, 1, 16, 2, 12, 8, 11, 10, 19, 4, 7, 17])
(1740556.0, [0, 1, 16, 2, 12, 6, 8, 11, 19, 10])
(1715879.0, [0, 1, 16, 2, 12, 6, 8, 11, 19, 10, 4])
(1740556.0, [0, 6, 16, 2, 1, 11, 12, 8, 4])
(1740557.0, [0, 6, 11, 16, 2, 1, 12, 8, 19, 4, 10, 17])
(1740558.0, [0, 6, 1, 2, 12, 11, 8, 16, 10, 19, 4, 13, 17])
(1740558.0, [0, 6, 2, 12, 1, 16, 11, 8, 19, 4, 10, 17])
(1740558.0, [0, 6, 2, 16, 12, 1, 8, 11, 10, 19, 4, 13, 17, 7])
(1740559.0, [0, 8, 16, 11])
(1740561.0, [0, 6, 1, 12, 2, 16, 17, 8, 11, 10])
(1740565.0, [0, 6, 8, 11, 1, 12, 16, 2, 17, 13])
(1740566.0, [0, 6, 2, 1, 16, 12, 11, 8, 10, 4, 17, 

In [22]:
best[1]

[0, 6, 1, 2, 16, 12, 8, 11, 10, 19, 4, 17, 13, 7, 14, 18, 3, 15, 9, 5, 0]

In [23]:
best

(2117963.0,
 [0, 6, 1, 2, 16, 12, 8, 11, 10, 19, 4, 17, 13, 7, 14, 18, 3, 15, 9, 5, 0])

In [24]:
nx.path_weight(G,best[1],'weight')

2117963.0

In [25]:
pd.read_csv('DATA/solutions.csv')

,Instance,Value
0,Atlanta,2003763
1,Berlin,7542
2,Boston,893536
3,Champaign,52643
4,Cincinnati,277952
5,Denver,100431
6,NYC,1555060
7,Philadelphia,1395981
8,Roanoke,655454
9,SanFrancisco,810196


In [26]:
nx.path_weight(G,best[1],'weight')/pd.read_csv('DATA/solutions.csv')['Value'][0]

1.056992768106807